# Explainer

Download files

In [3]:
import requests
import zipfile
import os
from pathlib import Path

# Create data directory if it doesn't exist
Path('data').mkdir(exist_ok=True)

# Download the zip file
url = "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
response = requests.get(url)

# Save the zip file temporarily
zip_path = "data/ml-25.zip"
with open(zip_path, 'wb') as f:
    f.write(response.content)

# Extract the contents to the data folder
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('data')

# Remove the temporary zip file
os.remove(zip_path)

In [7]:
from model.emf import EMFModel
from data_reader.data_reader import DataReader
from recommender.recommender import Recommender
from explainer.emf import EMFExplainer

## Load Data

In [2]:
train_data = DataReader('data/ml-100k/u.data',sep='\t',names=['userId','itemId','rating','timestamp'])

/Users/varunmallya/Documents/personal_projects/final_project/movie-recommender-project/explainability/data_reader/data_reader.py:30: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self._num_item = int(self._dataset[["itemId"]].nunique())
/Users/varunmallya/Documents/personal_projects/final_project/movie-recommender-project/explainability/data_reader/data_reader.py:31: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  self._num_user = int(self._dataset[["userId"]].nunique())


## Define EMF Model

In [4]:
emf = EMFModel(learning_rate=0.01,reg_term=0.001,expl_reg_term=0.0,latent_dim=80,epochs=10,positive_threshold=3,knn=10)

Fit the model

In [5]:
emf.fit(train_data)

  0%|          | 0/10 [00:00<?, ?it/s]

True

## Set up Recommender

In [6]:
recommender = Recommender(train_data, emf)
recommendations = recommender.recommend_all()

Recommending for users:   0%|          | 0/943 [00:00<?, ?it/s]

In [8]:
explanations = EMFExplainer(emf, recommendations, train_data)

In [ ]:
recommendations_with_explainations = explanations.explain_recommendations() 

## Checkout Recommendations

In [13]:
recommendations_with_explainations['userId'] = recommendations_with_explainations['userId'] + 1
recommendations_with_explainations['itemId'] = recommendations_with_explainations['itemId'] + 1
recommendations_with_explainations.head(10)

,userId,itemId,rank,explanations
1176,2.0,1450.0,1.0,{}
12,2.0,286.0,2.0,{5: 1}
201,2.0,475.0,3.0,"{3: 1, 4: 4, 5: 4}"
135,2.0,409.0,4.0,"{4: 2, 5: 4}"
1094,2.0,1368.0,5.0,{}
238,2.0,512.0,6.0,"{4: 2, 5: 2}"
384,2.0,658.0,7.0,{}
29,2.0,303.0,8.0,"{4: 1, 5: 2}"
374,2.0,648.0,9.0,{4: 1}
210,2.0,484.0,10.0,{5: 6}
